In [1]:
using BenchmarkTools
using NPZ
using JSON

Compile it with ifort, and check the exported symbols:
```
call "c:\Program Files (x86)\IntelSWTools\compilers_and_libraries_2019.5.281\windows\bin\ifortvars.bat" intel64
ifort -O2 /dll clipping.f90 -o ifort_clipping.dll
DUMPBIN /exports clipping.dll
```

If not on Windows, maybe add `-static-intel`, i.e.:
```
ifort -O2 /dll -static-intel clipping.f90 -o ifort_clipping.dll
```

Install MinGW for gfortran, make sure to link libgfortran statically or the DLL will not be accepted.
```
gfortran -O2 -shared -static-libgfortran clipping.f90 -o gfortran_clipping.dll
```

In [2]:
function ifort_area_of_intersection(a, b)
    ndim, nvertex, ntriangles = size(a)
    out = Vector{Float64}(undef, ntriangles)
    ccall(
        (:area_of_intersection, "./src/fortran/ifort_clipping.dll"),
        Nothing,
        (Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Float64}, Ptr{Float64}, Ptr{Float64}),
        Ref(ndim), Ref(nvertex), Ref(ntriangles), a, b, out,
    )
    return out
end

function gfortran_area_of_intersection(a, b)
    ndim, nvertex, ntriangles = size(a)
    out = Vector{Float64}(undef, ntriangles)
    ccall(
        (:area_of_intersection, "./src/fortran/gfortran_clipping.dll"),
        Nothing,
        (Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Float64}, Ptr{Float64}, Ptr{Float64}),
        Ref(ndim), Ref(nvertex), Ref(ntriangles), a, b, out,
    )
    return out
end

gfortran_area_of_intersection (generic function with 1 method)

In [3]:
a = permutedims(npzread("triangles_a.npy"), (3, 2, 1))
b = permutedims(npzread("triangles_b.npy"), (3, 2, 1))

2×3×10000000 Array{Float64,3}:
[:, :, 1] =
 0.772866  0.570218  0.250386
 0.609051  0.958381  0.258065

[:, :, 2] =
 0.264513  0.664046  0.124763
 0.538956  0.928762  0.803564

[:, :, 3] =
 0.47856   0.674365  0.393399
 0.301384  0.281799  0.901712

...

[:, :, 9999998] =
 0.398022  0.0983778  0.43207
 0.561544  0.233543   0.38628

[:, :, 9999999] =
 0.982461  0.107067  0.588445
 0.401328  0.310718  0.162313

[:, :, 10000000] =
 0.529699  0.942806  0.724126
 0.150737  0.965228  0.731719

In [4]:
times = Dict()
for s in [1, 10, 100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]
    aa = a[:, :, 1:s]
    bb = b[:, :, 1:s]
    res = @belapsed(ifort_area_of_intersection($aa, $bb))
    times[s] = res
end

open("timing/julia-ifort.json","w") do f
    JSON.print(f, times)
end

npzwrite("answers/julia-ifort.npy", ifort_area_of_intersection(a[:, :, 1:10_000], b[:, :, 1:10_000]))

In [5]:
times = Dict()
for s in [1, 10, 100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]
    aa = a[:, :, 1:s]
    bb = b[:, :, 1:s]
    res = @belapsed(gfortran_area_of_intersection($aa, $bb))
    times[s] = res
end

open("timing/julia-gfortran.json","w") do f
    JSON.print(f, times)
end

npzwrite("answers/julia-gfortran.npy", gfortran_area_of_intersection(a[:, :, 1:10_000], b[:, :, 1:10_000]))